In [ ]:
# from keras.preprocessing.image import ImageDataGenerator

# train_datagen = ImageDataGenerator(
#         rescale=1./255)
# #         shear_range=0.2,
# #         zoom_range=0.2,
# #         horizontal_flip=True)
        
# val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# train_image_generator = train_datagen.flow_from_directory('/media/monster/New Volume1/AerialImageDataset/train/train_frames/',
#     batch_size = 32,
#     target_size=(256, 256), 
#     color_mode='rgb', 
#     classes=None,
#     class_mode=None
# ) #NORMALLY 4/8/16/32)

# train_mask_generator = train_datagen.flow_from_directory(
# '/media/monster/New Volume1/AerialImageDataset/train/train_masks/',
#     batch_size = 32,
#     target_size=(256, 256), 
#     color_mode='rgb', 
#     classes=None,
#     class_mode=None
# ) #NORMALLY 4/8/16/32)

# val_image_generator = val_datagen.flow_from_directory(
# '/media/monster/New Volume1/AerialImageDataset/train/val_frames/',
#     batch_size = 32,
#     target_size=(256, 256), 
#     color_mode='rgb', 
#     classes=None,
#     class_mode=None) #NORMALLY 4/8/16/32)


# val_mask_generator = val_datagen.flow_from_directory(
# '/media/monster/New Volume1/AerialImageDataset/train/val_masks/',
#     batch_size = 32,
#     target_size=(256, 256), 
#     color_mode='rgb', 
#     classes=None,
#     class_mode=None
#     ) #NORMALLY 4/8/16/32)

In [ ]:
# train_generator = zip(train_image_generator, train_mask_generator)
# val_generator = zip(val_image_generator, val_mask_generator)

In [33]:
import cv2

def data_gen(img_folder, mask_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    random.shuffle(n)
    while (True):
        img = np.zeros((batch_size, 256, 256, 3)).astype('float')
        mask = np.zeros((batch_size, 256, 256, 1)).astype('float')

        for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 
            train_img = cv2.imread(img_folder+'/'+n[i])/255.
            train_img =  cv2.resize(train_img, (256, 256))# Read an image from folder and resize

            img[i-c] = train_img #add to array - img[0], img[1], and so on.


            train_mask = cv2.imread(mask_folder+'/'+n[i], cv2.IMREAD_GRAYSCALE)/255.
            train_mask = cv2.resize(train_mask, (256, 256))
            train_mask = train_mask.reshape(256, 256, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]

            mask[i-c] = train_mask

        c+=batch_size

        if(c+batch_size>=len(os.listdir(img_folder))):
            c=0
            random.shuffle(n)
                      # print "randomizing again"
        yield img, mask




train_frame_path = '/media/monster/New Volume1/AerialImageDataset/train/train_frames/train'
train_mask_path = '/media/monster/New Volume1/AerialImageDataset/train/train_masks/train'

val_frame_path = '/media/monster/New Volume1/AerialImageDataset/train/val_frames/val'
val_mask_path = '/media/monster/New Volume1/AerialImageDataset/train/val_masks/val'

# Train the model
train_gen = data_gen(train_frame_path,train_mask_path, batch_size = 4)
val_gen = data_gen(val_frame_path,val_mask_path, batch_size = 4)


In [30]:
import numpy as np 
import os
# import skimage.io as io
# import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet(pretrained_weights = None,input_size = (256,256,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [7]:
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy

smooth = 1e-9

# This is the competition metric implemented using Keras
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * (K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

# We'll construct a Keras Loss that incorporates the DICE score
def dice_loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1. - (2. * intersection + 1.) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.)


def bce_dice_loss(y_true, y_pred):
    return 0.5 * binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)


In [67]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import random
# import model

NO_OF_TRAINING_IMAGES = len(os.listdir('/media/monster/New Volume1/AerialImageDataset/train/train_frames/train'))
NO_OF_VAL_IMAGES = len(os.listdir('/media/monster/New Volume1/AerialImageDataset/train/val_frames/val'))

# print(NO_OF_TRAINING_IMAGES)
# print(NO_OF_VAL_IMAGES)

NO_OF_EPOCHS = 30

BATCH_SIZE = 32

weights_path = '/media/monster/New Volume1/AerialImageDataset/train/mod.h5'

m = unet()
# opt = Adam(lr=1E-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# m.compile(loss=bce_dice_loss,
#               optimizer=opt,
#               metrics=[dice_coef])

checkpoint = ModelCheckpoint(weights_path, monitor='accuracy', 
                             verbose=1, save_best_only=True, mode='max')

csv_logger = CSVLogger('./log.out', append=True, separator=';')

earlystopping = EarlyStopping(monitor = 'accuracy', verbose = 1,
                              min_delta = 0.01, patience = 3, mode = 'max')

callbacks_list = [checkpoint, csv_logger, earlystopping]

results = m.fit_generator(train_gen, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_gen, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE), 
                          callbacks=callbacks_list)
# m.save('Model.h5')

/home/monster/.conda/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=Tensor("in...)`


Epoch 1/30
3/3 [==============================] - 60s 20s/step - loss: 0.4766 - acc: 0.8448 - val_loss: 0.5052 - val_acc: 0.7953
Epoch 2/30


/home/monster/.conda/envs/tensorflow/lib/python2.7/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/home/monster/.conda/envs/tensorflow/lib/python2.7/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: acc,loss,val_acc,val_loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


2/3 [===================>..........] - ETA: 11s - loss: 0.5107 - acc: 0.8073

KeyboardInterrupt: 

In [18]:
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_145 (Conv2D)             (None, 256, 256, 64) 640         input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_146 (Conv2D)             (None, 256, 256, 64) 36928       conv2d_145[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_25 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_146[0][0]                 
__________________________________________________________________________________________________
conv2d_147

In [54]:
test = data_gen('/media/monster/New Volume1/AerialImageDataset/train/test_frames','/media/monster/New Volume1/AerialImageDataset/train/test_masks', batch_size = 1)

In [55]:
pred = m.predict_generator(test,18)

In [56]:
pred.shape

(18, 256, 256, 1)

In [57]:
pred[0].shape

(256, 256, 1)

In [63]:
from PIL import Image

img = Image.fromarray(pred[0], 'RGB')
# img.save('my.png')
img.show()